In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install gdown
!pip install transformers
!pip install ultralytics
!pip install pybktree
!pip install levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 130.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [3]:
%%capture
!pip install pyngrok
!pip uninstall protobuf -y
!pip install protobuf==3.20.*
!pip install streamlit
!pip install Pillow
%env PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python
!ngrok authtoken 2mdKor2sRARzy00rzHF4tijetfp_34n4ANTCXK9egrXRg4m3G


#!pip install googletrans==4.0.0-rc1  # Specific version of googletrans
!pip install streamlit-extras


In [4]:
%%capture
import streamlit as st


In [38]:
from pyngrok import ngrok

# Start an HTTP tunnel on the default Streamlit port (8501)
public_url = ngrok.connect(8501, "http")
print(f"Public URL: {public_url}")

Public URL: NgrokTunnel: "https://dbce-34-87-40-102.ngrok-free.app" -> "http://localhost:8501"


In [6]:
import transformers
from transformers import TrOCRProcessor

print(transformers.__version__)


4.51.3


In [12]:
import gdown
import os
# Create a Deployment directory


# Define download URLs using Google Drive file IDs
image_logo = 'https://drive.google.com/file/d/1JAiZS9_yXJ4QlBPHy9OrLINq3Er-3SK0/view?usp=sharing'
image_file = 'https://drive.google.com/file/d/1d6EKrhZQAVIcsbU2NSXXFEpXu2LNm9Zg/view?usp=drive_link'

# Download .py file into Deployment folder
gdown.download(image_logo,
               "team_logo.png", quiet=False,fuzzy=True)

# Download image file into Deployment folder
gdown.download(image_file,
               "logo_image.jpg", quiet=False,fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=1JAiZS9_yXJ4QlBPHy9OrLINq3Er-3SK0
To: /content/team_logo.png
100%|██████████| 71.4k/71.4k [00:00<00:00, 70.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1d6EKrhZQAVIcsbU2NSXXFEpXu2LNm9Zg
To: /content/logo_image.jpg
100%|██████████| 391k/391k [00:00<00:00, 84.4MB/s]


'logo_image.jpg'

In [37]:
%%writefile app.py
import streamlit as st
from PIL import Image
import os
import uuid
import os
import cv2
import gdown
import cProfile
import pstats
import streamlit as st
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from transformers.utils import logging
import os.path
import re
from ultralytics import YOLO
import matplotlib.pyplot as plt
import cv2
import pybktree
import Levenshtein
#################################
profiler = cProfile.Profile()

class RegionOCR:
    def __init__(self):
        """
        Initializes the PrescriptionOCR class.
        """
        self.local_path = './Backend/Resources/weights/arabic_ocr_model'
        if not os.path.exists(self.local_path):
            self.download_arabic_model()

        logging.set_verbosity_error()

        # Load the OCR models here
        self.english_processor = TrOCRProcessor.from_pretrained('microsoft/trocr-base-handwritten')
        self.english_model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-base-handwritten')

        self.arabic_processor = TrOCRProcessor.from_pretrained(self.local_path)
        self.arabic_model = VisionEncoderDecoderModel.from_pretrained(self.local_path)

    def extract_text(self, image, lang) -> str:
        """
        Extracts text from the given image.

        :param: image : The image to extract info from.

        Returns:
            str: The extracted text.
        """

        if lang == 1:
            return self.predict_eng(image)[0]
        elif lang == 0:
            return self.predict_ara(image)[0]
        else:
            return ''

    def predict_ara(self, image):
        pixel_values = self.arabic_processor(image, return_tensors="pt").pixel_values
        ids = self.arabic_model.generate(pixel_values)
        return self.arabic_processor.batch_decode(ids, skip_special_tokens=True)

    def predict_eng(self, image):

        pixel_values = self.english_processor(image, return_tensors="pt").pixel_values
        ids = self.english_model.generate(pixel_values)
        return self.english_processor.batch_decode(ids, skip_special_tokens=True)

    def download_arabic_model(self):
        """
        Downloads the Arabic OCR model if it does not exist locally.
        """
        os.makedirs(self.local_path, exist_ok=True)

        # Define the model files ID's
        config_id = "1-QuP1ZoXsAjJSkO2AVrXiSIdUhp3zww_"
        gen_config_id = "1-ORYC4N1liybcyuDihjGSNzWqV3oSH9J"
        model_tensor_id = "1-YlWSsZy0TGHwtU2eR7Ng78k0dYJuKm0"
        processor_id = "1-NjWDOpHa9QHh_W5xdu-u_pfoufj7-1K"
        sentence_piece_id = "1-WYclVqXwNxXMfRE5tN6K-VVjkPzyQq6"
        special_tokens_id = "1-HCqIsfz56dg0V-fBs8VzGVs92sPZg8L"
        tokenizer_config_id = "1-B65XNidxQNdZxW8QijJiQUmru6oHz9K"
        tokenizer_id = "1-8dheZMmlG879XGsGJiMgpvgNZot6QtN"

        # Download the model files
        gdown.download(f"https://drive.google.com/uc?id={config_id}",
                       os.path.join(self.local_path, 'config.json'),
                       quiet=False,
                       fuzzy=True)
        gdown.download(f"https://drive.google.com/uc?id={gen_config_id}",
                       os.path.join(self.local_path, 'generation_config.json'),
                       quiet=False,
                       fuzzy=True)
        gdown.download(f"https://drive.google.com/uc?id={model_tensor_id}",
                       os.path.join(self.local_path, 'model.safetensors'),
                       quiet=False,
                       fuzzy=True)
        gdown.download(f"https://drive.google.com/uc?id={processor_id}",
                       os.path.join(self.local_path, 'preprocessor_config.json'),
                       quiet=False,
                       fuzzy=True)
        gdown.download(f"https://drive.google.com/uc?id={sentence_piece_id}",
                       os.path.join(self.local_path, 'sentencepiece.bpe.model'),
                       quiet=False,
                       fuzzy=True)
        gdown.download(f"https://drive.google.com/uc?id={special_tokens_id}",
                       os.path.join(self.local_path, 'special_tokens_map.json'),
                       quiet=False,
                       fuzzy=True)
        gdown.download(f"https://drive.google.com/uc?id={tokenizer_config_id}",
                       os.path.join(self.local_path, 'tokenizer_config.json'),
                       quiet=False,
                       fuzzy=True)
        gdown.download(f"https://drive.google.com/uc?id={tokenizer_id}",
                       os.path.join(self.local_path, 'tokenizer.json'),
                       quiet=False,
                       fuzzy=True)



class RoiExtractor:
    """
    This class is responsible for extracting regions of interest (ROIs) from images using a YOLO model.
    It identifies specific regions in the image based on the model's predictions and categorizes them into
    different types such as language, frequency, and medication, then extract the text from them.
    """

    def __init__(self):
        """
        Initializes the RoiExtractor class.
        It downloads the YOLO model if it does not exist locally and loads the model.
        """
        self.local_path = './Backend/Resources/weights/Roi_extractor.pt'
        if not os.path.exists(self.local_path):
            self.download_model()
        self.yolo = YOLO(self.local_path)
        self.classes = self.yolo.names
        # print(self.classes.items())
        # Load the OCR model here
        self.ocr = RegionOCR()

    def extract_info(self, img_path):
        """
        Predicts the regions of interest in the given image using the YOLO model
        and gets the description associated with each one.

        :param img: the image which should be processed
        :return: parts of the image each one include the bounding box, the description of it and the predicted label
        """
        img = cv2.imread(img_path)
        img = cv2.resize(img, (640, 640))
        boxes = self.get_ROI(img)

        img = cv2.imread(img_path)
        return self.add_lang_type(boxes, img)

    def get_ROI(self, img):
        """
        This function is used to get the region of interest (ROI) from the image using the YOLO model.

        :param img: the image which should be processed
        :return: bounding boxes, confidence scores and class ids
        """
        # get the region of interest (ROI) from the image
        roi = self.yolo.predict(img, conf=0.5, verbose=False)
        w, h = img.shape[1], img.shape[0]

        # print(len(roi[0]))
        # plt.imshow(img)
        # plt.show()

        # plt.imshow(roi[0].plot())
        # plt.show()
        result = roi[0].obb
        result = sorted(result, key=lambda x: x.xyxy[0][1].cpu().numpy())  # sort according to the y-axis

        # get the bounding boxes, confidence scores and class ids
        conf = []
        xyxy = []
        cls_ids = []
        for res in result:
            cls_ids.append(int(res.cls[0].cpu().numpy()))
            conf.append(res.conf[0].cpu().numpy())

            x1, y1, x2, y2 = res.xyxy[0].cpu().numpy()
            x1, y1 ,x2, y2 = max(0, x1), max(0, y1), min(w, x2), min(h, y2)
            x1, y1, x2, y2 = x1/w, y1/h, x2/w, y2/h

            xyxy.append((x1, y1, x2, y2))

        return xyxy, conf, cls_ids

    def add_lang_type(self, boxes, img):
        """
        This function is used to add the language and type of the prescription to the bounding boxes.

        :param boxes: the bounding boxes, confidence scores and class ids
        :param img: the image which should be processed
        :return: parts of the image each one include the bounding box, the description of it and the predicted label
        """
        bbox, conf, cls_ids = boxes

        num_med = 0
        num_freq = 0
        num_lang = 0
        num_types = 0

        # count the number of each class
        for i in range(len(cls_ids)):
            if cls_ids[i] == 2:
                num_freq += 1
            elif cls_ids[i] == 3:
                num_med += 1

            if cls_ids[i] <= 1:
                num_lang += 1
            else:
                num_types += 1

        parts = []
        frequency_index = 0
        medicine_index = 0

        remove_list = []
        # TODO: check if the medicine number not equal to the frequency number or if prescription doesnt have a language
        for i in range(len(bbox)):
            best_iou, box_idx = 0, None
            for j in range(i + 1, len(bbox)):
                cur_iou = self.iou(bbox[i], bbox[j])
                if cur_iou > 0.5 and cur_iou > best_iou:  # extract the info when the boxes refer to the same region
                  best_iou, box_idx = cur_iou, j


            # for i in range(len(parts)):
            #   if self.iou(parts[i][0], bbox[i]) > 0.5:
            #     continue
            if best_iou != 0:
                lang, prescription_type = self.get_region_info(cls_ids[i], cls_ids[box_idx])

                # crop the region of interest
                x1, y1, x2, y2 = bbox[i]
                x1_1, y1_1, x2_1, y2_1 = int(x1*img.shape[1]), int(y1*img.shape[0]), int(x2*img.shape[1]), int(y2*img.shape[0])

                x1, y1, x2, y2 = bbox[box_idx]
                x1_2, y1_2, x2_2, y2_2 = int(x1*img.shape[1]), int(y1*img.shape[0]), int(x2*img.shape[1]), int(y2*img.shape[0])

                x1 = min(x1_1, x1_2)
                y1 = min(y1_1, y1_2)
                x2 = max(x2_1, x2_2)
                y2 = max(y2_1, y2_2)

                cropped_img = img[y1:y2, x1:x2]

                # get the label of the ROI
                if prescription_type == 'Medicine':
                    medicine_index += 1
                else:
                    frequency_index += 1

                # Use the OCR to get the text
                text = self.ocr.extract_text(cropped_img, lang)
                label = re.sub(r"[^\w\s]", "", text)
                label = label.strip()
                # plt.imshow(cropped_img)
                # plt.show()
                # print(label)
                # print("iou:", best_iou)
                # print(label, self.classes.get(prescription_type), self.classes.get(lang))
                # plt.imshow(cropped_img)
                # plt.show()
                # add the label with the other info into the list
                parts.append((bbox[i], (prescription_type, lang), label))
                    # break

            if frequency_index == num_freq and medicine_index == num_med:
                break

        return parts

    @staticmethod
    def iou(b1, b2):
        """
        This function is used to calculate the intersection over union (IoU) of two bounding boxes.

        :param b1: the first bounding box
        :param b2: the second bounding box
        :return: IOU of the two bounding boxes
        """
        x1, y1, x2, y2 = b1
        x3, y3, x4, y4 = b2

        # calculate the area of the two bounding boxes
        area1 = (x2 - x1) * (y2 - y1)
        area2 = (x4 - x3) * (y4 - y3)

        # get the intersection coordinates
        intersection_x1 = max(x1, x3)
        intersection_y1 = max(y1, y3)
        intersection_x2 = min(x2, x4)
        intersection_y2 = min(y2, y4)

        # calculate the area of the intersection
        intersection_area = max(0, intersection_x2 - intersection_x1) * max(0, intersection_y2 - intersection_y1)

        # calculate the area of the union
        union_area = area1 + area2 - intersection_area

        return intersection_area / union_area if union_area > 0 else 0

    def get_region_info(self, id1, id2):
        """
        This function is used to get the language and type of the prescription from the class ids.

        :param id1: the first class id
        :param id2: the second class id
        :return: language and prescription type of the region.
        """
        # get class names from the ids
        class_name1 = list(self.classes.keys())[int(id1)]
        class_name2 = list(self.classes.keys())[int(id2)]
        # extract language and whether the region has frequency or medication
        lang=None
        prescription_type = None
        if id1 <= 1:
            lang = class_name1
        else:
            prescription_type = class_name1

        if id2 <= 1:
            lang = class_name2
        else:
            prescription_type = class_name2

        return lang, prescription_type

    def download_model(self):
        """
        Downloads the YOLO model from Google Drive.
        It creates the necessary directories and downloads the model file.
        """
        os.makedirs(os.path.dirname(self.local_path), exist_ok=True)
        model_link = 'https://drive.google.com/file/d/1O1gQBJUdXgEw6Ttu7p_pi63--iQVTozN/view?usp=sharing'
        gdown.download(model_link, self.local_path, quiet=False, fuzzy=True)



class PrescriptionOCR:
    def __init__(self):
        self.roi_extractor = RoiExtractor()
        self.frequency_corpus_url = "https://drive.google.com/file/d/1HAGoD0C8PnyjrpTdONfZ_p2NbgdGkaoA/view?usp=drive_link"
        self.medicine_corpus_url = "https://drive.google.com/file/d/1JIKGrTHapxYicaqPHImJh74Vbt2h1the/view?usp=sharing"

        self.download_corpus()
        medicines, frequencies = self.read_wordlist()

        self.medicine_BK_tree = pybktree.BKTree(Levenshtein.distance, medicines)
        self.frequency_BK_tree = pybktree.BKTree(Levenshtein.distance, frequencies)

    def process_image(self, img_path: str) -> str:
        """
        Read the image and make any needed processing before extracting text from it.

        :return: preprocessed image.
        """
        img = cv2.imread(img_path)

        # Resize the image to a fixed size
        # img = cv2.resize(img, (1024, 1024))
        # TODO: add auto-orientation and de-skewing
        return img

    def predict(self, img_path: str) -> list:
        """
        Predict the text from the image.

        :return: predicted text.
        """
        # Read and process the image
        img = self.process_image(img_path)

        # Get ROI from the image
        image_parts = self.roi_extractor.extract_info(img_path)

        num_freq = 0
        num_med = 0
        last_type = None
        last_label = None
        pairs = []
        for part in image_parts:
            # Get the bounding box coordinates
            x1, y1, x2, y2 = part[0]
            # Get the ROI description
            prescription_type, lang = part[1]

            # Get the text from the image
            label = part[2]

            # get the nearst right word at a maximum distance of 3
            label, prescription_type = self.get_nearst_word_type(label, prescription_type, 5)
            # print(prescription_type, label)

            # Check if prescription type is empty
            if prescription_type == None:
                pairs.append([])
                continue
            # Check if the last type is not None
            if last_type is not None:
                # Check if the last type is different from the current type
                if last_type != prescription_type:
                    # pair is (medicine, frequency)
                    pair = (label, last_label) if prescription_type == 3 else (last_label, label) # 3 -> medicine
                    pairs.append(pair)

                    # change last type to None
                    last_type = None
                else:
                    # print("missing value and the current type:", prescription_type)
                    pair = (last_label, '') if prescription_type == 3 else ('', last_label)
                    pairs.append(pair)
                    last_label = label
            else:
                last_label = label
                last_type = prescription_type
        return pairs

    def get_nearst_word_type(self, word: str, prescription_type: str, max_distance: int = 3) -> (str, str):
        # get the nearst word to the given label from the appointment and the medicine corpus
        frequency = self.get_nearst_word(word, self.frequency_BK_tree, 3)
        medicine = self.get_nearst_word(word, self.medicine_BK_tree, 3)

        # check if the word is in the medicine or frequency corpus
        if medicine is not None and frequency is not None:
            # check which one is the nearest
            if medicine[0] < frequency[0]:
                label = medicine[1]
                prescription_type = 3  #  3 -> medicine
            elif frequency[0] < medicine[0]:
                label = frequency[1]
                prescription_type = 2  #  2 -> frequency
            else:  # if they are equal then check the prescription type
                label = medicine[1] if prescription_type == 3 else frequency[1]
        elif medicine is not None:
            label = medicine[1]
            prescription_type = 3
        elif frequency is not None:
            label = frequency[1]
            prescription_type = 2
        else:  # if not in corpus then return the word itself
            label = word
        return label, prescription_type

    def get_nearst_word(self, word: str, bk_tree, max_distance: int = 3) -> str | None:
        """
        Get the nearest word to the given word from the medicine corpus.

        :param word: The word to find the nearest word for.
        :param bk_tree: The BK-tree to search in.
        :param max_distance: The maximum distance to consider.
        :return: The nearest word.
        """
        nearest_word = bk_tree.find(word, max_distance)
        if len(nearest_word) > 0:
            return nearest_word[0]
        else:
            return None

    def download_corpus(self):
        if not os.path.exists("./Backend/Resources/frequency.txt"):
            # Download the appointment corpus
            frequency_corpus_local_path = "./Backend/Resources/frequency.txt"
            gdown.download(self.frequency_corpus_url, frequency_corpus_local_path, quiet=False, fuzzy=True)

        if not os.path.exists("./Backend/Resources/medicine.txt"):
            # Download the medicine corpus
            medicine_corpus_local_path = "./Backend/Resources/medicine.txt"
            gdown.download(self.medicine_corpus_url, medicine_corpus_local_path, quiet=False, fuzzy=True)

    def read_wordlist(self):
        """
        Read the medicine and frequency files and return a two list of words.
        """
        with open("./Backend/Resources/medicine.txt", "r") as f:
            medicines = f.read().splitlines()

        with open("./Backend/Resources/frequency.txt", "r") as f:
            frequencies = f.read().splitlines()
        return medicines, frequencies

################################
# Create a temporary folder to store uploaded images
TEMP_DIR = "temp_uploads"
os.makedirs(TEMP_DIR, exist_ok=True)

# Set Streamlit page config
st.set_page_config(page_title="PharmaLens", layout="centered")
# Custom CSS to make the sidebar background grey
st.markdown(
    """
    <style>
        /* Sidebar styling */
        [data-testid="stSidebar"] {
            background-color: #f0f0f0;
        }

        [data-testid="stSidebar"] h1,
        [data-testid="stSidebar"] h2,
        [data-testid="stSidebar"] h3,
        [data-testid="stSidebar"] p {
            color: #333;
        }

        /* File uploader label */
        [data-testid="stFileUploaderLabel"] {
            color: white !important;
            font-size: 16px !important;
        }

        /* Image caption override (if used) */
        .css-1hsbpi5 {
            color: white !important;
            font-size: 16px !important;
        }

        /* Submit button */
        button[kind="primary"] {
            color: white !important;
            font-size: 18px !important;
        }

        /* Extracted result text */
        .extracted-text {
            color: white !important;
            font-size: 20px !important;
        }

        .result-box {
            background-color: #2d2d2d;
            padding: 10px;
            border-radius: 8px;
            margin-bottom: 10px;
        }

        /* Success message styling */
        [data-testid="stAlert-success"] {
            background-color: #2d2d2d !important;
            color: white !important;
            font-size: 18px !important;
            border-left: 6px solid #1db954;
        }
    </style>
    """,
    unsafe_allow_html=True
)


# Title and Logo layout
col1, col2 = st.columns([4, 1])
with col1:
    st.title("PharmaLens🔬🔍")
with col2:
    st.image("logo_image.jpg", use_container_width=True)  # Make sure this exists in the working directory


# Sidebar with team name/logo
    with st.sidebar:
        #st.markdown("## 🧠 Team MedVision")
        st.write("AI-powered Prescription OCR System")
        # Optional: logo image if you have it
        st.image("team_logo.png", width=150)
        st.markdown("---")
        st.markdown("### 📇 Connect with Us")

        st.markdown(
            """
            <ul style='padding-left: 1em;'>
                <li><a href="https://www.linkedin.com/in/kareem-wael-k918/" target="_blank" style="text-decoration: none; color: #0e76a8;">Kareem Wael</a></li>
                <li><a href="https://www.linkedin.com/in/azza-hassan-40097a276/" target="_blank" style="text-decoration: none; color: #0e76a8;">Azza Hassan Said</a></li>
                <li><a href="https://www.linkedin.com/in/ahmed-hussein-el-shahat/" target="_blank" style="text-decoration: none; color: #0e76a8;">Ahmed Hussien</a></li>
            </ul>
            """,
            unsafe_allow_html=True
        )




# File uploader
st.markdown("<p style='color:white; font-size:16px;'>Upload a medical prescription image</p>", unsafe_allow_html=True)
uploaded_file = st.file_uploader("", type=["png", "jpg", "jpeg"])



# Define submit function from user
def submit(images):
    """
    Processes a list of prescription images and extracts structured prescription data.
    Args:
        images (list of str): List of file paths to input prescription images.
    Returns:
        list of list of tuples: Each inner list contains (medicine, frequency) pairs.
    """
    results = []
    ocr = PrescriptionOCR()
    for img_path in images:
        try:
            prescription_data = ocr.predict(img_path)
            results.append(prescription_data)
        except Exception as e:
            print(f"⚠️ Error processing image {img_path}: {e}")
            results.append([])
    return results

# Main logic
if uploaded_file is not None:
    image = Image.open(uploaded_file)
    st.image(image, caption=None, use_container_width=True)  # Hide default caption
    st.markdown("<p style='color:white; font-size:16px;'>Uploaded Image</p>", unsafe_allow_html=True)


    if st.button("🧾Submit Prescription"):
        # Save uploaded image temporarily
        with st.spinner("Extracting information..."):
                temp_filename = f"{uuid.uuid4().hex}.png"
                temp_path = os.path.join(TEMP_DIR, temp_filename)
                image.save(temp_path)

                # Call submit with the temp path
                all_results = submit([temp_path])  # List of 1 image

        st.success("✅ Extraction complete!")
        if all_results and all_results[0]:
            st.subheader("Extracted Information")
            for med, freq in all_results[0]:
              st.markdown(
                  f"""
                  <div class="result-box">
                      <div class="extracted-text">
                          <strong>Medicine:</strong> {med}<br>
                          <strong>Frequency dosage:</strong> {freq if freq else 'Not specified'}
                      </div>
                  </div>
                  """,
                  unsafe_allow_html=True
              )

        # Cleanup temp file
        os.remove(temp_path)


Overwriting app.py


In [ ]:
!streamlit run app.py &




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.87.40.102:8501

2025-05-01 19:42:59.041 Uncaught exception GET /_stcore/stream (127.0.0.1)
HTTPServerRequest(protocol='http', host='dbce-34-87-40-102.ngrok-free.app', method='GET', uri='/_stcore/stream', version='HTTP/1.1', remote_ip='127.0.0.1')
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/streamlit/web/bootstrap.py", line 347, in run
    if asyncio.get_running_loop().is_running():
       ^^^^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: no running event loop

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/tornado/websocket.py", line 938, in _accept_connection
    open_result = handler.open(*handler.open_args, **handler.open_kwargs)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^